In [ ]:
import os
import re
import imaplib

from dotenv import load_dotenv
from datetime import datetime

In [ ]:
from main import get_transactions_from_imap, save_transactions_to_file, export_transactions_text

In [ ]:
load_dotenv()

IMAP_USER_EMAIL_ADDRESS = os.getenv('IMAP_USER_EMAIL_ADDRESS')
IMAP_USER_PASSWORD = os.getenv('IMAP_USER_PASSWORD')
IMAP_SERVER = os.getenv('IMAP_SERVER')
IMAP_FILTER=os.getenv('IMAP_FILTER')

# Fetch the environment variable 'IMAP_BANK_MSG_STARTS' and split it by |.
# to get a list of start strings for bank messages. This is done to handle
# multiple possible start strings for different types of bank messages.
IMAP_BANK_MSG_STARTS = os.getenv('IMAP_BANK_MSG_STARTS').split('|')

IMAP_BANK_MSG_END=os.getenv('IMAP_BANK_MSG_END')
REGEX_BANK_FILE=os.getenv('REGEX_BANK_FILE')

In [ ]:
# this is done to make SSL connection with GMAIL
imap_con = imaplib.IMAP4_SSL(IMAP_SERVER)

# logging the user in
imap_con.login(IMAP_USER_EMAIL_ADDRESS, IMAP_USER_PASSWORD)

# calling function to check for email under this label
imap_con.select('Inbox')

(transactions_text, email_with_problems, emails_without_transactions) = get_transactions_from_imap(imap_con)

In [ ]:
emails_without_transactions

In [ ]:
notification_regex = []
with open(REGEX_BANK_FILE) as regex_bank_file:
    for line in regex_bank_file:
        notification_regex.append(line.strip('\n'))

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H:%M:%S")
csv_outputfile = f"target/transactions-{dt_string}.csv"
text_outputfile = f"target/transactions-{dt_string}.text"
save_transactions_to_file(transactions_text, text_outputfile)
export_transactions_text(transactions_text, notification_regex, "csv", csv_outputfile)